# Create Tasks
Tool to compute initial configurations for expert trajectory collection.

In [ ]:
'''imports'''
import os
from collections import deque
import itertools
import json
import csv
import pprint
from tqdm import tqdm

import numpy as np
import scipy.special

from geeco_gym.utils.grids import compute_grid_v2
from geeco_gym.utils.volumes import sample_point_within_sphere

In [ ]:
'''directory setup'''
root_path = os.environ['GEECO_ROOT']
# tmp
tmp_dir = os.path.join(root_path, 'notebooks', 'tmp')
notebook_name = 'dataset-create_tasks'
notebook_tmp_path = os.path.join(tmp_dir, notebook_name)
os.makedirs(notebook_tmp_path, exist_ok=True) 
print("Tmp path: %s" % (notebook_tmp_path, ))

In [ ]:
'''input'''
# scenario_name = 'pad1-cube1'
# scenario_name = 'pad1-cube2'
# scenario_name = 'pad2-cube1'
scenario_name = 'pad2-cube2'
# scenario_name = 'pad2-cube2-clutter4'
# scenario_name = 'pad2-cube2-clutter12'
# scenario_name = 'push-pad1-cube1'
# scenario_name = 'push-pad1-cube2'
# scenario_name = 'push-pad2-cube1'
# scenario_name = 'push-pad2-cube2'

scenario2task_configs = {
    'pad1-cube1' : {'num_obj_confs': 2000, 'task_multiplier': 2},
    'pad1-cube2' : {'num_obj_confs': 2000, 'task_multiplier': 1},
    'pad2-cube1' : {'num_obj_confs': 2000, 'task_multiplier': 1},
    'pad2-cube2' : {'num_obj_confs': 1000, 'task_multiplier': 1},
    'push-pad1-cube1' : {'num_obj_confs': 2000, 'task_multiplier': 2},
    'push-pad1-cube2' : {'num_obj_confs': 100, 'task_multiplier': 20},
    'push-pad2-cube1' : {'num_obj_confs': 100, 'task_multiplier': 20},
    'push-pad2-cube2' : {'num_obj_confs': 250, 'task_multiplier': 4},
    'pad2-cube2-clutter4' : {'num_obj_confs': 25, 'task_multiplier': 1},
    'pad2-cube2-clutter12' : {'num_obj_confs': 250, 'task_multiplier': 1},
}
task_configs = scenario2task_configs[scenario_name]
num_obj_confs = task_configs['num_obj_confs']  # x num. tasks
task_multiplier = task_configs['task_multiplier']  # how often is each task executed?


num_partitions = 4  # num. of init.csv partitions to create

# actuator config
gripper_mocap = 'robot0:mocap'  # qpos
scenario2gripper_xpos_0 = {
    'pad1-cube1' : [1.3419, 0.7491, 0.555],
    'pad1-cube2' : [1.3419, 0.7491, 0.555],
    'pad2-cube1' : [1.3419, 0.7491, 0.555],
    'pad2-cube2' : [1.3419, 0.7491, 0.555],
    'push-pad1-cube1' : [1.3419, 0.7491, 0.8],
    'push-pad1-cube2' : [1.3419, 0.7491, 0.8],
    'push-pad2-cube1' : [1.3419, 0.7491, 0.8],
    'push-pad2-cube2' : [1.3419, 0.7491, 0.8],
    'pad2-cube2-clutter4' : [1.3419, 0.7491, 0.555],
    'pad2-cube2-clutter12' : [1.3419, 0.7491, 0.555],
}
gripper_sample_radius = 0.03

# workspace dimensions
scenario2dims = {
    'pad1-cube1': {'minmax_x': (1.075, 1.425), 'minmax_y': (0.35, 1.15), 'offset_z': 0.27, 'tiling_xy': (6, 8), 'goal_offset_x': 0},
    'pad1-cube2' : {'minmax_x': (1.075, 1.425), 'minmax_y': (0.35, 1.15), 'offset_z': 0.27, 'tiling_xy': (6, 8), 'goal_offset_x': 0},
    'pad2-cube1' : {'minmax_x': (1.075, 1.425), 'minmax_y': (0.35, 1.15), 'offset_z': 0.27, 'tiling_xy': (4, 7), 'goal_offset_x': 0},
    'pad2-cube2' : {'minmax_x': (1.075, 1.425), 'minmax_y': (0.35, 1.15), 'offset_z': 0.27, 'tiling_xy': (4, 7), 'goal_offset_x': 0},
    'push-pad1-cube1' : {'minmax_x': (1.2, 1.3), 'minmax_y': (0.45, 1.05), 'offset_z': 0.27, 'tiling_xy': (6, 8), 'goal_offset_x': 0.1},
    'push-pad1-cube2' : {'minmax_x': (1.175, 1.4), 'minmax_y': (0.5, 1), 'offset_z': 0.27, 'tiling_xy' : (2, 3), 'goal_offset_x' : 0.125},
    'push-pad2-cube1' : {'minmax_x': (1.175, 1.4), 'minmax_y': (0.5, 1), 'offset_z': 0.27, 'tiling_xy' : (2, 3), 'goal_offset_x' : 0.125},
    'push-pad2-cube2' : {'minmax_x': (1.175, 1.4), 'minmax_y': (0.5, 1), 'offset_z': 0.27, 'tiling_xy' : (2, 3), 'goal_offset_x' : 0.125},
    'pad2-cube2-clutter4' : {'minmax_x': (1.075, 1.425), 'minmax_y': (0.35, 1.15), 'offset_z': 0.27, 'tiling_xy': (4, 7), 'goal_offset_x': 0},
    'pad2-cube2-clutter12' : {'minmax_x': (1.075, 1.425), 'minmax_y': (0.35, 1.15), 'offset_z': 0.27, 'tiling_xy': (4, 7), 'goal_offset_x': 0},
}

In [ ]:
'''compute spawn locations for objects'''

# object names
scenario2objects = {
    'pad1-cube1' : ['object0:joint', 'goal0:joint'],
    'pad1-cube2' : ['object0:joint', 'object1:joint', 'goal0:joint'],
    'pad2-cube1' : ['object0:joint', 'goal0:joint', 'goal1:joint'],
    'pad2-cube2' : ['object0:joint', 'object1:joint', 'goal0:joint', 'goal1:joint'],
    'push-pad1-cube1' : ['object0:joint', 'goal0:joint'],
    'push-pad1-cube2' : ['object0:joint', 'object1:joint', 'goal0:joint'],
    'push-pad2-cube1' : ['object0:joint', 'goal0:joint', 'goal1:joint'],
    'push-pad2-cube2' : ['object0:joint', 'object1:joint', 'goal0:joint', 'goal1:joint'],
    'pad2-cube2-clutter4' : [
            'object0:joint', 'object1:joint', 'goal0:joint', 'goal1:joint', 'clutter0:joint',                    'clutter1:joint', 'clutter2:joint', 'clutter3:joint'
    ],
    'pad2-cube2-clutter12' : [
            'object0:joint', 'object1:joint', 'goal0:joint', 'goal1:joint',
            'clutter0:joint', 'clutter1:joint', 'clutter2:joint', 'clutter3:joint',
            'clutter4:joint', 'clutter5:joint', 'clutter6:joint', 'clutter7:joint',
            'clutter8:joint', 'clutter9:joint', 'clutter10:joint', 'clutter11:joint',
    ],
}

# compute centers
dims = scenario2dims[scenario_name]
centers = compute_grid_v2(dims['minmax_x'], dims['minmax_y'], dims['tiling_xy'])
num_points = len(centers)
num_objects = len(scenario2objects[scenario_name])
print("Number of spawn cells: %d" % num_points)
print("Number of objects to spawn: %d" % num_objects)
num_max_spawn_pos = scipy.special.comb(num_points, num_objects) * scipy.special.factorial(num_objects)
print("Max. number of spawn positions (ordered): %d" % num_max_spawn_pos)

In [ ]:
# compute all possible spawn tuples
grid_idx = list(range(num_points))

# sample spawn combinations
index_tuples = []
for _ in tqdm(range(num_obj_confs)):
    while True:  # rejection sampling, ordering disregarded
        t = tuple(np.random.choice(num_points, num_objects, replace=False))
        if not t in index_tuples:
            index_tuples.append(t)
            break

# orig_idx = deque(grid_idx)
# index_tuples = []
# for _ in range(1, num_points, num_objects-1):
#     rotated_indices = []
#     for _ in range(num_objects-1):
#         orig_idx.rotate(1)
#         rotated_indices.append(list(orig_idx))
#     index_tuples.extend(list(zip(grid_idx, *rotated_indices)))

num_spawn_pos = len(index_tuples)
print("Number of spawn positions:", num_spawn_pos)

# compute all object & goal permutations
goal_names = [n.rstrip(':joint') for n in scenario2objects[scenario_name] if n.startswith('goal')]
cube_names = [n.rstrip(':joint') for n in scenario2objects[scenario_name] if n.startswith('object')]
task_tuples = []
for goal in goal_names:
    for cube in cube_names:
        task_tuples.append((goal, cube))
# apply task multiplier
task_tuples = task_tuples * task_multiplier
num_tasks = len(task_tuples)
print("Number of task configurations:", num_tasks)
        
# multiply spawn positions with task setups
# repeat each index_tuple by num_tasks
index_tuples = [[t] * num_tasks for t in index_tuples]
index_tuples = list(itertools.chain(*index_tuples))
# repeat task_tuples num_spawn_pos times
task_tuples = task_tuples * num_spawn_pos

# sample initial gripper positions for each setup
gripper_tuples = []
for _ in range(len(index_tuples)):
    rnd_xpos = np.array(scenario2gripper_xpos_0[scenario_name]) + np.array(sample_point_within_sphere(gripper_sample_radius))
    gripper_tuples.append((rnd_xpos[0], rnd_xpos[1], rnd_xpos[2], 1, 0, 1, 0))

print("Number of repeated index tuples:", len(index_tuples))
print("Number of repeated task tuples:", len(task_tuples))
print("Number of gripper qpos0 tuples:", len(gripper_tuples))

In [ ]:
# chunk index_tuples to be distributed across multiple files
index_chunks = list(np.split(np.array(index_tuples), num_partitions))
pprint.pprint([chunk.shape for chunk in index_chunks])
gripper_chunks = list(np.split(np.array(gripper_tuples), num_partitions))
pprint.pprint([chunk.shape for chunk in gripper_chunks])
task_chunks = list(np.split(np.array(task_tuples), num_partitions))
pprint.pprint([chunk.shape for chunk in task_chunks])

In [ ]:
# export spawn positions as csv (chunks)
cur_idx = 0
for idx_tuples, grp_tuples, tsk_tuples in zip(index_chunks, gripper_chunks, task_chunks):
    idx_tuples = list(idx_tuples)
    tsk_tuples = list(tsk_tuples)
    cur_idx += len(idx_tuples)
    csv_file = 'init-%s-%04d.csv' % (scenario_name, cur_idx)
    csv_path = os.path.join(notebook_tmp_path, csv_file)
    print(csv_path)
    with open(csv_path, 'w', newline='') as fp:
        writer = csv.writer(fp, delimiter=';')
        # write header for joints
        header_row = [['%s::px' % n, '%s::py' % n, '%s::pz' % n, '%s::qw' % n, '%s::qx' % n, '%s::qy' % n, '%s::qz' % n] for n in scenario2objects[scenario_name]]
        header_row = list(itertools.chain(*header_row))  # flatten
        # append gripper header
        gripper_header = [['%s::px' % n, '%s::py' % n, '%s::pz' % n, '%s::qw' % n, '%s::qx' % n, '%s::qy' % n, '%s::qz' % n] for n in [gripper_mocap]]
        gripper_header = list(itertools.chain(*gripper_header))  # flatten
        header_row += gripper_header
        # append task header
        header_row += ['task::goal', 'task::object']
        idx_goal0_x = header_row.index('goal0:joint::px') if 'goal0:joint::px' in header_row else -1
        idx_goal1_x = header_row.index('goal1:joint::px') if 'goal1:joint::px' in header_row else -1
        writer.writerow(header_row)
        for idx_t, grp_t, tsk_t in zip(idx_tuples, grp_tuples, tsk_tuples):
            init_pos = [[centers[i][0], centers[i][1], dims['offset_z'], 1, 0, 0, 0] for i in idx_t]  # qpos_0 as list from center coordinates for each object
            init_pos = list(itertools.chain(*init_pos))  # flatten list of lists into single row vector
            if idx_goal0_x != -1:
                init_pos[idx_goal0_x] += dims['goal_offset_x']
            if idx_goal1_x != -1:
                init_pos[idx_goal1_x] += dims['goal_offset_x']
            # append gripper qpos
            init_pos += list(grp_t)
            # append task definition
            init_pos += [tsk_t[0], tsk_t[1]]
            writer.writerow(init_pos)

In [ ]:
# export spawn positions as csv
csv_file = 'init-%s.csv' % (scenario_name, )
csv_path = os.path.join(notebook_tmp_path, csv_file)
print(csv_path)
with open(csv_path, 'w', newline='') as fp:
    writer = csv.writer(fp, delimiter=';')
    # write header for joints
    header_row = [['%s::px' % n, '%s::py' % n, '%s::pz' % n, '%s::qw' % n, '%s::qx' % n, '%s::qy' % n, '%s::qz' % n] for n in scenario2objects[scenario_name]]
    header_row = list(itertools.chain(*header_row))  # flatten
    # append gripper header
    gripper_header = [['%s::px' % n, '%s::py' % n, '%s::pz' % n, '%s::qw' % n, '%s::qx' % n, '%s::qy' % n, '%s::qz' % n] for n in [gripper_mocap]]
    gripper_header = list(itertools.chain(*gripper_header))  # flatten
    header_row += gripper_header
    # append task header
    header_row += ['task::goal', 'task::object']
    idx_goal0_x = header_row.index('goal0:joint::px') if 'goal0:joint::px' in header_row else -1
    idx_goal1_x = header_row.index('goal1:joint::px') if 'goal1:joint::px' in header_row else -1
    writer.writerow(header_row)
    for idx_t, grp_t, tsk_t in zip(index_tuples, gripper_tuples, task_tuples):
        init_pos = [[centers[i][0], centers[i][1], dims['offset_z'], 1, 0, 0, 0] for i in idx_t]  # qpos_0 as list from center coordinates for each object
        init_pos = list(itertools.chain(*init_pos))  # flatten list of lists into single row vector
        if idx_goal0_x != -1:
            init_pos[idx_goal0_x] += dims['goal_offset_x']
        if idx_goal1_x != -1:
            init_pos[idx_goal1_x] += dims['goal_offset_x']
        # append gripper qpos
        init_pos += list(grp_t)
        # append task definition
        init_pos += [tsk_t[0], tsk_t[1]]
        writer.writerow(init_pos)

In [ ]:
# read spawn positions from file
with open(csv_path) as fp:
    reader = csv.reader(fp, delimiter=';')
    iterator = iter(reader)
    header_row = next(iterator)
    pprint.pprint(header_row)
    while True:
        try:
            row = next(iterator)
        except StopIteration:
            break
        qpos_row = [float(e) for e in row[:-2]]
        task_row = [str(e) for e in row[-2:]]
        print(qpos_row, task_row)